# Natural Map

## imports

In [1]:
import JordanAlgebra
from sympy import Symbol, Matrix, sqrt, Pow, Add, Rational, postorder_traversal, factor, simplify, nsimplify, latex
from codegen.funcodegen import funcodegen
import numpy as np


## symbols

In [2]:
from JordanAlgebra import epsilon
from friction import mu, rn, rt1, rt2, un, ut1, ut2, xn, xt1, xt2, yn, yt1, yt2, u, r, x, y, ut


## The natural map function

In [3]:
Fnat = y - JordanAlgebra.projection(y - x)


## Raw jacobians

In [4]:
A_ = Fnat.jacobian(u)
B_ = Fnat.jacobian(r)

In [5]:
lambda1, lambda2, u1, u2 = JordanAlgebra.spectral_decomposition(y - x)

#def load(v):

#    with open(v) as r_file:
#        return pickle.load(r_file)

#A = load('Fnat_A')
#B = load('Fnat_B')
#Fnat = load('Fnat')
#Rnow = load('Fnat_Rnow')

In [6]:
from sympy import Piecewise
def add_split(expr, pos, f1, f2):
    n_expr = expr
    if type(expr) == Add:       
       n_expr1 = Add(*expr.args[:pos])
       n_expr2 = Add(*expr.args[pos:])
       n_expr = Add(f2(n_expr2), f1(n_expr1))
    return n_expr

def add_splitp(p_expr, lpos, f1, f2):
    if hasattr(p_expr, 'is_Piecewise') and p_expr.is_Piecewise:
        return Piecewise(*[(add_split(c[0], lpos[i], f1, f2), c[1]) for i, c in enumerate(p_expr.args)])
    else:
        return add_split(p_expr, lpos[0], f1, f2)

def add_splitv(mat, lpos, f1, f2):
    return Matrix(mat.shape[0], mat.shape[1], 
        lambda i,j: add_splitp(mat[i, j], lpos[i], f1, f2))

def map_matrix(mat, fun, args):
    return Matrix(mat.shape[0], mat.shape[1], 
                  lambda i,j: fun(mat[i, j], *(args[i])))



In [7]:
add_splitp(Piecewise((1 + ut1**2/2 + ut2**2/2, ut1>0), (ut1, ut1<=0)), [1,0], factor, factor)

#map_matrix(Matrix([1 + ut1**2/2 + ut2**2/2]), add_split, [1, factor, factor])

Piecewise(((ut1**2 + ut2**2)/2 + 1, ut1 > 0), (ut1, ut1 <= 0))

In [8]:
from sympy import init_printing
init_printing()

from sympy import Symbol
nut = Symbol(r'\lVert {u_{t}} \rVert', negative=False)
nrt = Symbol(r'\lVert {r_{t}} \rVert', negative=False)
lbd1 = Symbol(r'\lambda_1')
lbd2 = Symbol(r'\lambda_2')
delta1 = Symbol(r'\Delta_1')
delta2 = Symbol(r'\Delta_2')
ndelta = Symbol(r'\lVert {\Delta} \rVert', negative=False)
lepsilon = Symbol(r'\epsilon')
lut1 = Symbol(r'u_{t1}')
lut2 = Symbol(r'u_{t2}')
lun = Symbol(r'u_n')
lrt1 = Symbol(r'r_{t1}')
lrt2 = Symbol(r'r_{t2}')
lrn = Symbol(r'r_n')

from codegen.funcodegen import flatten_piecewise


def make_readable(expr, deep=False):
    return flatten_piecewise(expr).subs(lrt1, rt1).subs(lrt2, rt2).subs(lrn, rn).subs(lut1, ut1).subs(lut2, ut2).subs(lun, un).subs('rt1', rt1).subs('rt2', rt2).subs('rn', rn).subs('ut1', ut1).subs('ut2', ut2).subs('un', un).subs(lambda1, lbd1).subs(lambda2, lbd2).subs(ut1**2+ut2**2,nut**2).subs(rt1**2+rt2**2, nrt**2).subs(mu*ut2-rt2, delta2).subs(mu*ut1-rt1, delta1).subs(delta1**2+delta2**2, ndelta**2).subs(rt1, lrt1).subs(rt2, lrt2).subs(rn, lrn).subs(ut1, lut1).subs(ut2, lut2).subs(un, lun).subs(epsilon, lepsilon).subs(0.5, Rational(1, 2)).subs(sqrt(nrt**2), nrt).subs(sqrt(nut**2), nut).subs(sqrt(ndelta**2), ndelta).subs(sqrt(sqrt(nrt**2)), sqrt(nrt))


ndelta, sqrt((mu*ut2-rt2)**2+(mu*ut1-rt1)**2)

In [9]:
from sympy import expand
add_splitv(make_readable(Fnat), [[2], [1,0], [1,1]], factor, factor)


⎡                                  Max(0, \lambda_1) + Max(0, \lambda_2)      
⎢                          μ⋅r_n - ─────────────────────────────────────      
⎢                                                    2                        
⎢                                                                             
⎢⎧  \Delta₁⋅(Max(0, \lambda_1) - Max(0, \lambda_2))                           
⎢⎪- ─────────────────────────────────────────────── + r_{t1}  for \lVert {\Del
⎢⎨              2⋅\lVert {\Delta} \rVert                                      
⎢⎪                                                                            
⎢⎩                          r_{t1}                            for \lVert {\Del
⎢                                                                             
⎢⎧  \Delta₂⋅(Max(0, \lambda_1) - Max(0, \lambda_2))                           
⎢⎪- ─────────────────────────────────────────────── + r_{t2}  for \lVert {\Del
⎢⎪              2⋅\lVert {\Delta} \rVert            

In [10]:
P=make_readable(Fnat[1]).args[0][0]



In [11]:
add_split(make_readable(Fnat[2]), 2, factor, factor)

⎧  \Delta₂⋅Max(0, \lambda_1)   \Delta₂⋅Max(0, \lambda_2)                      
⎪- ───────────────────────── + ───────────────────────── + r_{t2}  for \lVert 
⎪   2⋅\lVert {\Delta} \rVert    2⋅\lVert {\Delta} \rVert                      
⎨                                                                             
⎪                  Max(0, \lambda_1)   Max(0, \lambda_2)                      
⎪         r_{t2} + ───────────────── - ─────────────────           for \lVert 
⎩                          2                   2                              

                          
{\Delta} \rVert > \epsilon
                          
                          
                          
{\Delta} \rVert ≤ \epsilon
                          

In [12]:
with open('codegen/tex/fnat.tex', 'w') as f:
    f.write(latex(add_splitv(make_readable(Fnat), [[2], [1,0], [1,1]], factor, factor)))

In [13]:
from codegen.maple import limzero, Maple, set_maple
maple = Maple(server='bizet.inria.fr')
set_maple(maple)
t = Symbol('t', positive=True, real=True)
maple_assumes = 'mu >= 0 and mu < 1 and rn >=0 and t > 0 and epsilon > 0'
maple.assume(maple_assumes)


No remote temporary directory (option server_tmpdir) specified, using /tmp/ on bizet.inria.fr


In [14]:
def nsimp(expr):
    n_expr = expr
    for subexpr in postorder_traversal(expr):
        n_expr = n_expr.subs(subexpr, nsimplify(subexpr))
    return n_expr

In [15]:
A=Matrix(3, 3, lambda i, j: 0)

###### ====================
A[0, 0]
====================

In [18]:
factor(make_readable(A_[0, 0]))

Heaviside(\lambda₁) + Heaviside(\lambda₂)
─────────────────────────────────────────
                    2                    

In [0]:
A[0, 0] = A_[0, 0]

In [20]:
with open('codegen/tex/A00_fnat.tex', 'w') as f:
     f.write(latex(factor(make_readable(A_[0, 0]))))

###### ====================
A[0, 1]
====================

In [21]:
factor(make_readable(A_[0, 1]))

μ⋅(\Delta₁⋅\lVert {u_{t}} \rVert⋅Heaviside(\lambda₁) - \Delta₁⋅\lVert {u_{t}} 
──────────────────────────────────────────────────────────────────────────────
                                                                              

\rVert⋅Heaviside(\lambda₂) + \lVert {\Delta} \rVert⋅u_{t1}⋅Heaviside(\lambda₁)
──────────────────────────────────────────────────────────────────────────────
    2⋅\lVert {\Delta} \rVert⋅\lVert {u_{t}} \rVert                            

 + \lVert {\Delta} \rVert⋅u_{t1}⋅Heaviside(\lambda₂))
─────────────────────────────────────────────────────
                                                     

In [28]:
make_readable(nsimp(limzero(A_[0, 1], [(ut1, 0), (ut2, 0)])))

  μ⋅r_{t1}⋅Heaviside(-\lVert {r_{t}} \rVert + μ⋅r_n - u_n)   μ⋅r_{t1}⋅Heavisid
- ──────────────────────────────────────────────────────── + ─────────────────
                  2⋅\lVert {r_{t}} \rVert                                    2

e(\lVert {r_{t}} \rVert + μ⋅r_n - u_n)
──────────────────────────────────────
⋅\lVert {r_{t}} \rVert                

pour \Delta_n différent de 0, ||u_t|| -> 0 (==> rt != 0)

deltan -> 0

In [23]:
from codegen.maple import mlimit
make_readable(nsimp(mlimit(A_[0, 1].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0)))

μ⋅r_{t1}⋅Heaviside(-\lVert {r_{t}} \rVert + μ⋅r_n - u_n)
────────────────────────────────────────────────────────
                 \lVert {r_{t}} \rVert                  

deltan -> 0 && ut -> 0 ==> rt -> 0

In [24]:
make_readable(nsimp(limzero(A_[0, 1], [(ut1, t), (ut2, t), (rt1, t), (rt2, t)])))


  ___                         
╲╱ 2 ⋅μ⋅Heaviside(μ⋅r_n - u_n)
──────────────────────────────
              2               

In [40]:
from sympy import And
A[0, 1] = Piecewise(
     (limzero(A_[0, 1], [(ut1, t), (ut2, t), (rt1, t), (rt2, t)]), And(sqrt(ut1**2+ut2**2) <= epsilon, sqrt(rt1**2+rt2**2) <= epsilon)),
     (mlimit(A_[0, 1].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0), sqrt((mu*ut1-rt1)**2+(mu*ut2-rt2)**2) <= epsilon),
     (A_[0, 1], And(sqrt(ut1**2+ut2**2) > epsilon, sqrt((mu*ut1-rt1)**2+(mu*ut2-rt2)**2) > epsilon)),
     (limzero(A_[0, 1], [(ut1, 0), (ut2, 0)]), And(sqrt((mu*ut1-rt1)**2+(mu*ut2-rt2)**2) > epsilon, sqrt(ut1**2+ut2**2) <= epsilon, sqrt(rt1**2+rt2**2) > epsilon)))
     
nsimp(make_readable(A[0, 1]))

⎧                                                          ___                
⎪                                                        ╲╱ 2 ⋅μ⋅Heaviside(μ⋅r
⎪                                                        ─────────────────────
⎪                                                                      2      
⎪                                                                             
⎪                                           μ⋅r_{t1}⋅Heaviside(-\lVert {r_{t}}
⎪                                           ──────────────────────────────────
⎪                                                            \lVert {r_{t}} \r
⎪                                                                             
⎨  ⎛        \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                       ⎛ 
⎪  ⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambda₁)   ⎜─
⎪  ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                       ⎝\
⎪- ─────────────────────────────────────────────────

In [41]:
with open('codegen/tex/A01_fnat.tex', 'w') as f:
     f.write(latex(factor(make_readable(A[0, 1]))))

###### ====================
A[0, 2]
====================

In [0]:
make_readable(A_[0,2])

deltan != 0, ut -> 0

In [0]:
make_readable(nsimp(limzero(A_[0, 2], [(ut1, t), (ut2, t)])))

In [0]:
make_readable(nsimp(limzero(A_[0, 2], [(ut1, t), (ut2, t), (rt1, t), (rt2, t)])))

In [0]:
make_readable(nsimp(mlimit(A_[0, 2].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0)))

###### ====================
A[1, 0]
====================

In [0]:
make_readable(A_[1, 0])

In [0]:
mlimit(A_[1, 0].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0)

###### ====================
A[1, 1]
====================

In [0]:
factor(make_readable(A_[1, 1]))

In [0]:
make_readable(nsimp(limzero(A_[1, 1], [(ut1, t), (ut2, t)])))

###### ====================
A[1, 2]
====================

In [0]:
make_readable(A_[1, 2])

In [0]:
make_readable(nsimp(limzero(A_[1,2],  [(ut1, t), (ut2, t)])))

###### ====================
A[2, 2]
====================

In [0]:
make_readable(limzero(A_[2, 2], [(ut1, 0), (ut2, t)]))

In [0]:
nsimplify(1.4142135623730951)